# Summary

Originally was going to work on auto cli here and did some planning re interface. Ended up working mostly on fallback decorator, isstatic, etc.

In [1]:
%load_ext autoreload
%autoreload 3

In [4]:
import gc
import inspect
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path

from htools import *

In [3]:
cd_root()

Current directory: /Users/hmamin/incendio


/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
print(
"""
# OPTION 1
@auto_cli()
def main():
    x = f1(a, b, c)
    x = f2(x=x, y=y, **f2_kwargs)
    print(x)

if __name__ == '__main__':
    fire.Fire(main)
    
    
# OPTION 2
@auto_cli(f1, f2)
def main():
    x = f1(a, b, c)
    x = f2(x=x, y=y, **f2_kwargs)
    print(x)

if __name__ == '__main__':
    fire.Fire(main)
"""
)


# OPTION 1
@auto_cli()
def main():
    x = f1(a, b, c)
    x = f2(x=x, y=y, **f2_kwargs)
    print(x)

if __name__ == '__main__':
    fire.Fire(main)
    
    
# OPTION 2
@auto_cli(f1, f2)
def main():
    x = f1(a, b, c)
    x = f2(x=x, y=y, **f2_kwargs)
    print(x)

if __name__ == '__main__':
    fire.Fire(main)



## isstatic

In [609]:
from inspect import ismethod, signature, getmembers, getmodule
import inspect
from types import FunctionType, MethodType

In [81]:
def bar():
    return 'in bar'

In [114]:
def baz(cls):
    return 'in baz'

In [135]:
def bak(unk):
    return 'in bak'

In [495]:
class FooBar:
    
    def walk(self, a):
        return 'a'
    
    @staticmethod
    def static_walk(x, y):
        return x*y

In [496]:
class Foo:
    
    class_var = 'my_class_var'
    
    def __init__(self, inst_var='my_inst_var'):
        self.inst_var = inst_var
        self.inst_var_func = bar
        self.inst_var_func_baz = baz
        self.inst_var_func_bak = bak
        self.inst_var_inspect = inspect.signature
        self.inst_var_foobar = FooBar().walk
        self.inst_var_foobar_static = FooBar.static_walk
        
    def my_method(self, x):
        return 'in regular method'
    
    @staticmethod
    def my_static(y):
        return 'in staticmethod', y
    
    @classmethod
    def my_classmethod(cls, *args):
        return cls('-'.join(args))
    
    @abstractmethod
    def my_abstract(self):
        raise NotImplementedError()
        
    @property
    def my_prop(self):
        return 'my_prop_val'

In [500]:
f = Foo()

In [501]:
f.inst_var_func

<function __main__.bar()>

In [502]:
f.my_static

<function __main__.Foo.my_static(y)>

In [503]:
hdir(Foo)

{'class_var': 'attribute',
 'my_abstract': 'method',
 'my_classmethod': 'method',
 'my_method': 'method',
 'my_prop': 'attribute',
 'my_static': 'method'}

In [504]:
hdir(f)

{'class_var': 'attribute',
 'inst_var': 'attribute',
 'inst_var_foobar': 'method',
 'inst_var_foobar_static': 'method',
 'inst_var_func': 'method',
 'inst_var_func_bak': 'method',
 'inst_var_func_baz': 'method',
 'inst_var_inspect': 'method',
 'my_abstract': 'method',
 'my_classmethod': 'method',
 'my_method': 'method',
 'my_prop': 'attribute',
 'my_static': 'method'}

In [505]:
type(f.my_static)

function

In [521]:
def _isstatic(meth):
    # Logic relies on convention of reserving "cls" and "self" as names of the
    # first argument of classmethods and standard methods, respectively, and
    # not using them for other arguments.
    if (ismethod(meth) or '.' not in getattr(meth, '__qualname__', '')
        or not callable(meth)): 
        return False
    params_ = list(params(meth))
    if not params_ or params_[0] in {'cls', 'self'}:
        return False
    return True

In [626]:
def method_of(obj):
    cls, name = obj.__qualname__.split('.')
    return dict(getmembers(getmodule(obj)))[cls]

In [655]:
assert method_of(Foo.my_static) == method_of(f.my_static) == Foo

In [628]:
method_of(f.my_static)

__main__.Foo

In [653]:
def _isstatic2(meth):
    # First check isn't required but I want to avoid reaching the hackier bits
    # of code if necessary. This catches regular methods and attributes.
    if ismethod(meth) or not callable(meth): return False
    parts = getattr(meth, '__qualname__', '').split('.')
    if len(parts) != 2: return False
    cls = method_of(meth)
    return hasstatic(cls, parts[-1])

In [674]:
def has_classmethod(cls, meth_name):
    return isinstance(getattr_static(cls, meth_name), classmethod)

In [675]:
def is_classmethod(meth):
    if not ismethod(meth): return False
    parts = getattr(meth, '__qualname__', '').split('.')
    if len(parts) != 2: return False
    cls = method_of(meth)
    return has_classmethod(cls, parts[-1])

In [676]:
# OBJ (attrs)
for k, v in hdir(f).items():
    if v == 'method': continue   
    print(k, v)
    obj = getattr(f, k)
    print(obj)
    print(type(obj))
    print('qualname dots', getattr(obj, '__qualname__', '').count('.'))
    print()

class_var attribute
my_class_var
<class 'str'>
qualname dots 0

inst_var attribute
my_inst_var
<class 'str'>
qualname dots 0

my_prop attribute
my_prop_val
<class 'str'>
qualname dots 0



In [677]:
# OBJ (methods)
for k, v in hdir(f).items():
    if v == 'attribute': continue  
    obj = getattr(f, k)
    print(k, obj)
    
#     print('type', type(obj))
#     print('qualname', getattr(obj, '__qualname__', ''))
#     print('is method:', ismethod(obj))
#     print('is FunctionType:', isinstance(obj, FunctionType))
#     print('is MethodType:', isinstance(obj, MethodType))
#     print(list(params(obj)))  

#     print('is static', _isstatic(obj))
#     print('static 2', _isstatic2(obj))

    print('has classmethod', has_classmethod(f, k))
    print('is classmethod', is_classmethod(obj))
    
    print()

inst_var_foobar <bound method FooBar.walk of <__main__.FooBar object at 0x11ce4da90>>
has classmethod False
is classmethod False

inst_var_foobar_static <function FooBar.static_walk at 0x1a23e3b950>
has classmethod False
is classmethod False

inst_var_func <function bar at 0x1a237e09d8>
has classmethod False
is classmethod False

inst_var_func_bak <function bak at 0x1a22bc39d8>
has classmethod False
is classmethod False

inst_var_func_baz <function baz at 0x1a23104ea0>
has classmethod False
is classmethod False

inst_var_inspect <function signature at 0x10fc2dd08>
has classmethod False
is classmethod False

my_abstract <bound method Foo.my_abstract of <__main__.Foo object at 0x11ce4dd30>>
has classmethod False
is classmethod False

my_classmethod <bound method Foo.my_classmethod of <class '__main__.Foo'>>
has classmethod True
is classmethod True

my_method <bound method Foo.my_method of <__main__.Foo object at 0x11ce4dd30>>
has classmethod False
is classmethod False

my_static <functio

In [665]:
isinstance(Foo.__dict__['my_classmethod'], classmethod)

True

In [666]:
isinstance(Foo.my_classmethod, classmethod)

False

In [678]:
# CLASS (attrs)
for k, v in hdir(Foo).items():
    if v == 'method': continue  
    obj = getattr(Foo, k)
    print(k, obj)
    
#     print('type', type(obj))
#     print('qualname', getattr(obj, '__qualname__', ''))
#     print('is method:', ismethod(obj))
#print('is FunctionType:', isinstance(obj, FunctionType))
#     print('is MethodType:', isinstance(obj, MethodType))
#     print(list(params(obj)))  

#     print('is static', _isstatic(obj))
#     print('is static2', _isstatic2(obj))
    
    print('has classmethod', has_classmethod(Foo, k))
    print('is classmethod', is_classmethod(obj))
    print()

class_var my_class_var
has classmethod False
is classmethod False

my_prop <property object at 0x11cdfc8b8>
has classmethod False
is classmethod False



In [679]:
# CLASS (methods)
for k, v in hdir(Foo).items():
    if v == 'attribute': continue  
    obj = getattr(Foo, k)
    print(k, obj)
    
#     print('type', type(obj))
#     print('qualname', getattr(obj, '__qualname__', ''))
#     print('is method:', ismethod(obj))
#     print('is FunctionType:', isinstance(obj, FunctionType))
#     print('is MethodType:', isinstance(obj, MethodType))
#     print(list(params(obj)))  

#     print('is static', _isstatic(obj))
#     print('is static 2', _isstatic2(obj))

    print('has classmethod', has_classmethod(Foo, k))
    print('is classmethod', is_classmethod(obj))
    print()

my_abstract <function Foo.my_abstract at 0x1a23b5b840>
has classmethod False
is classmethod False

my_classmethod <bound method Foo.my_classmethod of <class '__main__.Foo'>>
has classmethod True
is classmethod True

my_method <function Foo.my_method at 0x1a23b5bd08>
has classmethod False
is classmethod False

my_static <function Foo.my_static at 0x1a23b5b8c8>
has classmethod False
is classmethod False



In [642]:
list(params(Foo.my_static))

['y']

In [56]:
staticmethod??

In [563]:
hasstatic(Foo, 'my_static')

True

In [564]:
hasstatic(f, 'my_static')

True

## Override_self

Thinking out loud: we can't use add_kwargs here to update the method's signature because we don't know what attributes self has until the method is called (we get them by examining args[0] in wrapper). Any way around this would probably be extremely hacky. I suppose if we mandated `keep` we could add those ahead of time but we wouldn't be able to show defaults.

In [241]:
from contextlib import contextmanager

/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [153]:
@auto_repr
class Tree:
    
    def __init__(self, a, b, c=3):
        self.a = a
        self.b = b
        self.c = c
        
    def call(self, **kwargs):
        a = kwargs.pop('a', self.a)
        b = kwargs.pop('b', self.b)
        c = kwargs.pop('c', self.c)
        return a, b, c

In [154]:
t = Tree(1, 2, 3)
t

Tree(a=1, b=2, c=3)

In [155]:
t.call()

(1, 2, 3)

In [156]:
t.call(c=44)

(1, 2, 44)

@auto_repr
class Tree:
    
    def __init__(self, a, b, c=3):
        self.a = a
        self.b = b
        self.c = c
        
    @kwargs_fallback
    def call(self, **kwargs):
        # Signature shows a, b, and c with class default values.
        return a, b, c

In [242]:
@contextmanager
def temporary_global_scope(kwargs):
    # Make kwargs temporarily available as global vars.
    old_globals = globals().copy()
    globals().update(kwargs)
    try:
        yield
    finally:
        for k in kwargs:
            if k in old_globals:
                globals()[k] = old_globals[k]
            else:
                del globals()[k]

In [435]:
# diff name for now since htools has other kwargs_fallback func.
def fallback(meth=None, *, keep=(), drop=(), save=False):
    """Make instance/class attributes available as default arguments for a 
    method. Kwargs can be passed in to override one or more of them. You can
    also choose for kwargs to update the instance attributes if desired.
    
    When using default values for keep/drop/save, the decorator can be used
    without parentheses. If you want to change one or more arguments, they
    must be passed in as keyword args (meth is never explicitly passed in, of
    course).
    
    Parameters
    ----------
    meth: method
        The method to decorate. Unlike the other arguments, this is passed in
        implicitly.
    keep: Iterable[str] or str
        Name(s) of instance attributes to include. If you specify a value 
        here, ONLY these instance attributes will be made available as 
        fallbacks. If you don't pass in any value, the default is for all
        instance attributes to be made available. You can specify `keep`, 
        `drop`, or neither, but not both. This covers all possible options:
        keep only a few, keep all BUT a few, or keep all (drop all is the
        default case and doesn't require a decorator).
    drop: Iterable[str] or str
        Name(s) of instance attributes to ignore. I.e. if you want to make
        all instance attributes available as fallbacks except for self.df,
        you could specify drop=('df').
    save: bool
        If True, kwargs that share names with instance attributes will be
        overwritten with their new values. E.g. if we previously had
        self.lr = 3e-3 and you call your decorated method with 
        obj.mymethod(lr=1), self.lr will be set to 1.
    """
    if meth is None: 
        # Want to avoid errors if user passes in string or leaves comma out of
        # tuple when specifying keep/drop.
        return partial(fallback, keep=tolist(keep), drop=tolist(drop), 
                       save=save)
    
    @wraps(meth)
    def wrapper(*args, **kwargs):
        self = args[0]
        self_kwargs = vars(self)
        if keep or drop: self_kwargs = select(self_kwargs, keep, drop)

        # Update kwargs with instance attribute defaults. Also update self if
        # user asked to save kwargs.
        for k, v in self_kwargs.items():
            if k not in kwargs:
                kwargs[k] = v
            elif save:
                setattr(self, k, kwargs[k])
                
        # Execute and return.
        with temporary_global_scope(kwargs):
            return meth(*args, **kwargs)
    return wrapper

In [436]:
@auto_repr
class Tree:
    
    def __init__(self, a, b, c=3):
        self.a = a
        self.b = b
        self.c = c
        
    @fallback(save=True)
    def call(self, **kwargs):
        return a, b, c

In [437]:
t = Tree(1, 2, 3)
t.call(b='bbb')

(1, 'bbb', 3)

In [438]:
assert t.b == 'bbb'

In [439]:
@auto_repr
class Tree:
    
    def __init__(self, a, b, c=3):
        self.a = a
        self.b = b
        self.c = c
        
    @fallback(save=False)
    def call(self, **kwargs):
        return a, b, c

In [440]:
t = Tree(1, 2, 3)
t.call(b='bbb')

(1, 'bbb', 3)

In [441]:
assert t.b == 2

In [442]:
@auto_repr
class Tree:
    
    def __init__(self, a, b, c=3):
        self.a = a
        self.b = b
        self.c = c
        
    @fallback
    def call(self, **kwargs):
        return a, b, c

In [443]:
t = Tree(1, 2, 3)
t.call(b='bbb')

(1, 'bbb', 3)

In [444]:
assert t.b == 2

In [445]:
@auto_repr
class Tree:
    
    def __init__(self, a, b, c=3):
        self.a = a
        self.b = b
        self.c = c
        
    @fallback(keep=('a'))
    def call(self, **kwargs):
        return a, b, c

In [446]:
t = Tree(1, 2, 3)
with assert_raises(NameError):
    t.call(a='aaa', b='bbb')

As expected, got NameError(name 'c' is not defined).


In [460]:
@auto_repr
class Tree:
    
    def __init__(self, a, b, c=3):
        """testing
        
        Parameters
        ----------
        a: int
        b: int
        c: int
        """
        self.a = a
        self.b = b
        self.c = c
        
    @add_docstring(Tree.__init__)
    @fallback(drop=('c'))
    def call(self, **kwargs):
        return a, b

In [461]:
t = Tree(1, 2, 3)
t.call(b='bbb')

(1, 'bbb')

In [455]:
assert t.b == 2